# Przygotowanie danych

- Merge danych z różnych stacji
- Wybór kolumn
- Uzupełnianie braków
- Feature engineering wspólny

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="whitegrid")
DATA_DIR = Path("data")
STATIONS = {
    "Wroclaw": "12424",  # Cel
    "Legnica": "12415",  # Zachód
    "Opole": "12530",  # Wschód
    "Poznan": "12330",  # Północ
    "Klodzko": "12520",  # Południe
}
MAIN_STATION = "Wroclaw"

In [2]:
START_DATE="2018-01-01" # Z dużymi brakami w deszczu, ale pozostałe kolumny w większości są
# START_DATE="2022-05-26" # Pełne dane deszczu
START_DATE_FOR_RAIN_PREDICTIONS = "2022-05-26"

## Dane do prognozy temperatury

- Uzupełnianie ewentualnych braków
- Łączenie danych z różnych stacji
- Podstawowy feature engineering

In [3]:
COLS_MAIN = ['temp', 'rhum', 'prcp', 'snwd', 'wdir', 'wspd', 'wpgt', 'pres', 'cldc', 'coco']
COLS_NEIGHBOR = ['temp', 'pres', 'prcp', 'wspd', 'rhum', 'wdir']
def load_data():
    dfs = []
    for name, station_id in STATIONS.items():
        file_path = DATA_DIR / f"{name.lower()}.csv"
        df = pd.read_csv(file_path, parse_dates=['time'], index_col='time')
        df = df.loc[START_DATE:]

        # Select columns
        target_cols = COLS_MAIN if name == MAIN_STATION else COLS_NEIGHBOR
        available = [c for c in target_cols if c in df.columns]
        df = df[available]

        # Rename columns for neighbors
        if name != MAIN_STATION:
            df.columns = [f"{col}_{name}" for col in df.columns]
        
        dfs.append(df)

    return dfs

In [4]:
dfs = load_data()

In [5]:
def index_gap_report(df, freq="h"):
    idx = df.index.sort_values()

    expected = pd.date_range(
        start=idx.min(),
        end=idx.max(),
        freq=freq
    )

    missing = expected.difference(idx)

    # długości dziur
    diffs = idx.to_series().diff()
    step = pd.to_timedelta(1, unit=freq)

    gap_sizes = diffs[diffs > step]

    if len(gap_sizes) == 0:
        max_gap = pd.Timedelta(0)
    else:
        max_gap = gap_sizes.max()

    print(f"  missing timestamps: {len(missing)}")
    print(f"  longest gap: {max_gap}")

    return missing, gap_sizes

In [6]:
for df in dfs:
  index_gap_report(df)

  missing timestamps: 2
  longest gap: 0 days 03:00:00
  missing timestamps: 16
  longest gap: 0 days 03:00:00
  missing timestamps: 8
  longest gap: 0 days 03:00:00
  missing timestamps: 3
  longest gap: 0 days 03:00:00
  missing timestamps: 8
  longest gap: 0 days 03:00:00


In [7]:
def enforce_continuous_index(df, freq="h"):
    full_idx = pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq=freq
    )
    return df.reindex(full_idx)


In [8]:
dfs_new = []
for df in dfs:
  dfs_new.append(enforce_continuous_index(df))
dfs=dfs_new
for df in dfs:
  index_gap_report(df)

  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00


In [9]:
for df in dfs:
    print(df.isna().mean() * 100)

temp     0.002853
rhum     0.004279
prcp    55.700735
snwd    99.736110
wdir     0.029955
wspd     0.002853
wpgt     7.233436
pres     0.004279
cldc     3.126738
coco     4.484702
dtype: float64
temp_Legnica     0.022823
pres_Legnica     0.022823
prcp_Legnica    55.700735
wspd_Legnica     0.155481
rhum_Legnica     0.022823
wdir_Legnica     0.022823
dtype: float64
temp_Opole     0.011411
pres_Opole     0.011411
prcp_Opole    55.700735
wspd_Opole     0.128379
rhum_Opole     0.011411
wdir_Opole     0.011411
dtype: float64
temp_Poznan     0.004279
pres_Poznan     0.005706
prcp_Poznan    55.700735
wspd_Poznan     0.004279
rhum_Poznan     0.044219
wdir_Poznan     0.005706
dtype: float64
temp_Klodzko     0.011411
pres_Klodzko     0.011411
prcp_Klodzko    55.700735
wspd_Klodzko     0.232508
rhum_Klodzko     0.011411
wdir_Klodzko     0.159760
dtype: float64


Zasady uzupełniania:

- **temp** - jak godzinę temu
- **rhum** - jak godzinę temu
- **prcp** - 0
- **snwd** - 0
- **wdir** - jak godzinę temu
- **wspd** - jak godzinę temu
- **wpgt** - jak **wspd**
- **pres** - jak godzinę temu
- **tsun** - usuń całą kolumnę (zrobione wcześniej)
- **cldc** - jak godzinę temu
- **coco** - jak godzinę temu, a braki na początku danych 0

In [10]:
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # reguły bazowe (bez sufiksów)
    ffill_cols = ['temp', 'rhum', 'wdir', 'wspd', 'pres', 'cldc', 'coco']
    zero_cols = ['prcp', 'snwd', 'coco']

    # 1. zerowanie opadu i śniegu
    for base in zero_cols:
        cols = [c for c in df.columns if c == base or c.startswith(f"{base}_")]
        df[cols] = df[cols].fillna(0)

    # 2. forward fill zmiennych wolnozmiennych
    for base in ffill_cols:
        cols = [c for c in df.columns if c == base or c.startswith(f"{base}_")]
        df[cols] = df[cols].ffill()

    # 3. wpgt = jak wspd
    # stacja główna
    if 'wpgt' in df.columns and 'wspd' in df.columns:
        df['wpgt'] = df['wpgt'].fillna(df['wspd'])

    return df


In [11]:
dfs_new = []
for df in dfs:
  dfs_new.append(fill_missing_values(df))
dfs=dfs_new
for df in dfs:
    print(df.isna().mean() * 100)

temp    0.0
rhum    0.0
prcp    0.0
snwd    0.0
wdir    0.0
wspd    0.0
wpgt    0.0
pres    0.0
cldc    0.0
coco    0.0
dtype: float64
temp_Legnica    0.0
pres_Legnica    0.0
prcp_Legnica    0.0
wspd_Legnica    0.0
rhum_Legnica    0.0
wdir_Legnica    0.0
dtype: float64
temp_Opole    0.0
pres_Opole    0.0
prcp_Opole    0.0
wspd_Opole    0.0
rhum_Opole    0.0
wdir_Opole    0.0
dtype: float64
temp_Poznan    0.0
pres_Poznan    0.0
prcp_Poznan    0.0
wspd_Poznan    0.0
rhum_Poznan    0.0
wdir_Poznan    0.0
dtype: float64
temp_Klodzko    0.0
pres_Klodzko    0.0
prcp_Klodzko    0.0
wspd_Klodzko    0.0
rhum_Klodzko    0.0
wdir_Klodzko    0.0
dtype: float64


In [12]:
# Merge all stations on time index
df_combined = pd.concat(dfs, axis=1).sort_index()
df_combined.dropna()
print(df_combined.isna().mean() * 100)

temp            0.0
rhum            0.0
prcp            0.0
snwd            0.0
wdir            0.0
wspd            0.0
wpgt            0.0
pres            0.0
cldc            0.0
coco            0.0
temp_Legnica    0.0
pres_Legnica    0.0
prcp_Legnica    0.0
wspd_Legnica    0.0
rhum_Legnica    0.0
wdir_Legnica    0.0
temp_Opole      0.0
pres_Opole      0.0
prcp_Opole      0.0
wspd_Opole      0.0
rhum_Opole      0.0
wdir_Opole      0.0
temp_Poznan     0.0
pres_Poznan     0.0
prcp_Poznan     0.0
wspd_Poznan     0.0
rhum_Poznan     0.0
wdir_Poznan     0.0
temp_Klodzko    0.0
pres_Klodzko    0.0
prcp_Klodzko    0.0
wspd_Klodzko    0.0
rhum_Klodzko    0.0
wdir_Klodzko    0.0
dtype: float64


In [13]:
index_gap_report(df_combined)

  missing timestamps: 0
  longest gap: 0 days 00:00:00


(DatetimeIndex([], dtype='datetime64[ns]', freq='h'),
 Series([], Freq: h, dtype: timedelta64[ns]))

In [14]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    idx = df.index
    df['year'] = idx.year
    df['month'] = idx.month
    df['day'] = idx.day
    df['day_of_year'] = idx.dayofyear

    return df


In [15]:
df = feature_engineering(df_combined)
df.head()

,temp,rhum,prcp,snwd,wdir,wspd,wpgt,pres,cldc,coco,...,temp_Klodzko,pres_Klodzko,prcp_Klodzko,wspd_Klodzko,rhum_Klodzko,wdir_Klodzko,year,month,day,day_of_year
2018-01-01 00:00:00,10.1,72.0,0.0,0.0,220.0,18.0,18.0,1004.3,7.0,0.0,...,7.0,1006.9,0.0,10.8,81.0,230.0,2018,1,1,1
2018-01-01 01:00:00,10.1,71.0,0.0,0.0,220.0,18.0,18.0,1004.1,7.0,0.0,...,6.6,1006.9,0.0,7.2,80.0,230.0,2018,1,1,1
2018-01-01 02:00:00,9.8,71.0,0.0,0.0,220.0,14.4,14.4,1003.9,7.0,0.0,...,7.3,1006.2,0.0,10.8,77.0,200.0,2018,1,1,1
2018-01-01 03:00:00,10.1,69.0,0.0,0.0,230.0,21.6,21.6,1003.6,7.0,0.0,...,6.3,1006.0,0.0,18.0,79.0,170.0,2018,1,1,1
2018-01-01 04:00:00,10.3,68.0,0.0,0.0,220.0,21.6,21.6,1003.1,7.0,0.0,...,6.2,1006.1,0.0,18.0,82.0,190.0,2018,1,1,1


In [16]:
df.to_csv(DATA_DIR / "combined.csv", index_label="time")

In [ ]:
df = df.loc[START_DATE_FOR_RAIN_PREDICTIONS:]
df.to_csv(DATA_DIR / "combined_rain.csv", index_label="time")
